In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("datascience-to-SQL.ipynb")

## Conversion Notebook: From Data 8X's `datascience` library to SQL

You have now learned how to think computationally, how to express your thoughts in a modern programming language, Python, clearly and succinctly, and how to apply this knowledge to various tasks in Data Science.  In doing so, you have come to understand what it means to *interpret* a program.  The Python interpreter is a piece of software that executes - i.e., interprets - python programs.

In this notebook, we are going to build upon the concepts taught in Data 8X and utilize your understanding of `Table`s to have an intro crash course to learn a language called SQL. SQL is a language that relates to databases - most databases use SQL to query.  Formally, it performs operations defined in relational algebra. 

SQL is one of the most commonly used database languages. Many companies and businesses use SQL as a way to access and manipulate the data that is stored in their databases. 

SQL is a **declarative** language which means that you say *what* you want, not *how* to get it. We will be writing SQL queries to describe the table we want to output. This is different to Python for example, where we write code that explains how to get something. SQL queries describe the output of what we want. 

We will use a particular relational database management system, sqlite3, which actually stores the database in a conventional file.

In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## SQL Query Syntax 

The main syntax of the SQL language consists of one very long statement.

```
SELECT [DISTINCT] <column expression list>
   FROM <table>
   [WHERE <predicate>]
   [GROUP BY <column list>]
   [HAVING <predicate>]
   [ORDER BY <column list>]
   [LIMIT <integer>];
```

In SQL, queries and subqueries produce a new table, as do most `Table` methods.

Here's a pretty [good reference](https://www.dofactory.com/sql/syntax).  There are lots of them out there. [Here's another](http://www.zentut.com/sql-tutorial/).

SQL keywords (SELECT, FROM, WHERE, etc in the query template above) are case insensitive, but convention is to use uppercase for clarity.  Column names and text values are case sensitive.

For students who have come through Data8, the datascience package is very similar to SQL in terms of syntax and logic to SQL.

Let's start with a `Table` example.

In [2]:
# Table constructor method

# An example of creating a Table from a list of rows.
Table(["Flavor","Color","Price"]).with_rows([
    ('strawberry','pink', 3.55),
    ('chocolate','light brown', 4.75),
    ('chocolate','dark brown', 5.25),
    ('strawberry','pink',5.25),
    ('bubblegum','pink',4.75),
    ('chocolate','dark brown', 5.25)
])

# Select

The basic aspect of SQL is the select statement, which returns a table.

The following line of code creates a one-row Table
```
SELECT "strawberry" AS Flavor, "pink" AS Color, 3.55 AS Price; 
```

Note - column names are always text and have no `"` around them.

To try this out, run `sqlite3` creating a database file, say `testsql`

`sqlite3 testsql`

This will run interactively, so you can see the results of the queries you perform.

```
ideas/sql> sqlite3 testsql.db
SQLite version 3.13.0 2016-05-18 10:57:30
Enter ".help" for usage hints.
sqlite> select "strawberry" as Flavor, "pink" as Color, 3.55 as Price;
strawberry|pink|3.55
sqlite>
```
Don't forget the ";"


## Table as Union of tables

Creating a multi-row table (object) with a union of `select` statements
```
select "strawberry" as Flavor, "pink" as Color, 3.55 as Price union
select "chocolate","light brown", 4.75 union
select "chocolate","dark brown", 5.25 union
select "strawberry","pink",5.25 union
select "bubblegum","pink",4.75 union
select "strawberry","pink",5.25 union
select "chocolate","dark brown", 5.25;
```

Notice how a SQL table is a relation (meaning that each select statement above is a tuple of information that is stored as a row in the table), it is not a list.  

```
sqlite> select "strawberry" as Flavor, "pink" as Color, 3.55 as Price union
   ...> select "chocolate","light brown", 4.75 union
   ...> select "chocolate","dark brown", 5.25 union
   ...> select "strawberry","pink",5.25 union
   ...> select "bubblegum","pink",4.75 union
   ...> select "strawberry","pink",5.25 union
   ...> select "chocolate","dark brown", 5.25;
bubblegum|pink|4.75
chocolate|dark brown|5.25
chocolate|light brown|4.75
strawberry|pink|3.55
strawberry|pink|5.25
```
Notice, no duplicates.  

# Create table

Let's create a version of the table that might occur in a retail setting.  We have added a column for the transaction ID.
```
create table cones as
    select 1 as Id, "strawberry" as Flavor, "pink" as Color, 3.55 as Price union
    select 2, "chocolate","light brown", 4.75 union
    select 3, "chocolate","dark brown", 5.25 union
    select 4, "strawberry","pink",5.25 union
    select 5, "bubblegum","pink",4.75 union
    select 6, "chocolate", "dark brown", 5.25;
```

#### sqlite3

Paste that into your `sqlite3` session.  Then you can access the data.

```
sqlite> .headers on
sqlite> select * from cones;
ID|Flavor|Color|Price
1|strawberry|pink|3.55
2|chocolate|light brown|4.75
3|chocolate|dark brown|5.25
4|strawberry|pink|5.25
5|bubblegum|pink|4.75
6|chocolate|dark brown|5.25
sqlite> 
```

In [3]:
# Create a Table and assign it to a variable

cones = Table(["Id", "Flavor","Color","Price"]).with_rows([
    (1, 'strawberry','pink', 3.55),
    (2, 'chocolate','light brown', 4.75),
    (3, 'chocolate','dark brown', 5.25),
    (4, 'strawberry','pink',5.25),
    (5, 'bubblegum','pink',4.75),
    (6, 'chocolate','dark brown', 5.25)
])
cones

## Tables in a database

The sqlite3 command line provides pseudo-ops (which are special types of commands) to explore the tables in the database. In the following code chunks, we can see what the existing tables are and the schema of our database (schema is the structure of our database and describes how the tables are created and with what columns).

```
sqlite> .tables
cones
sqlite> .schema cones
CREATE TABLE cones(
  Id,
  Flavor,
  Color,
  Price
);
```
SQL also provides table management operations, besides `CREATE`.

```
sqlite> drop table cones;
sqlite> .tables
```

## Creating a table and its schema

While it is possible to create a table and fill it in, as in the example above, in most database settings tables are permanent objects that are created and then repeatedly accessed and modified.

To create an empty table and specify its schema:

```
CREATE TABLE cones (Id INT, Flavor TEXT, Color TEXT, Price REAL);
```
Then it can be filled in with `INSERT` statements.

## Insert

To insert a row into a table, you need to specify three things:

* The table, which you want to insert a new row, in the INSERT INTO clause.
* a comma-separated list of columns in the table surrounded by parentheses.
* a comma-separated list of values surrounded by parentheses in the VALUES clause.

```
INSERT INTO table(column1, column2,...)
VALUES (value1, value2,...);
```
For example, to fill out our cones table creation:

```
INSERT INTO cones VALUES (1, "strawberry", "pink", 3.55);
INSERT INTO cones VALUES (2, "chocolate","light brown", 4.75);
INSERT INTO cones VALUES (3, "chocolate","dark brown", 5.25);
INSERT INTO cones VALUES (4, "strawberry","pink",5.25);
INSERT INTO cones VALUES (5, "bubblegum","pink",4.75);
INSERT INTO cones VALUES (6, "chocolate", "dark brown", 5.25);
```

Individual columns-value relationships can be specified too.

```
insert into cones(Id, Flavor, Color, Price) values (7, "Vanila", "White", 3.95);
```

Columns where data is not inserted are filled with `null`.

```
sqlite> insert into cones(Flavor, Price) values ("Vanila", 2.25);
sqlite> select * from cones;
Id|Flavor|Color|Price
1|strawberry|pink|3.55
2|chocolate|light brown|4.75
3|chocolate|dark brown|5.25
4|strawberry|pink|5.25
5|bubblegum|pink|4.75
6|chocolate|dark brown|5.25
7|Vanila|White|3.95
|Vanila||2.25

```

In [4]:
cones.append((7, "Vanila", "White", 3.95))
cones

## Ordering

In addition to the set of records in a table, one often would like to order them in a particular away.  You've seen this with `Table.sort`.

In [5]:
# Tables have an explicit order, which can be controlled
cones.sort('Price')

The `select` statement has an `ORDER BY` clause. We write "ORDER BY" and then the column that we would like to sort our table by. Automatically, SQL chooses to sort in ascending order but we can do descending by specifying `DESC` after our column name.

```
SELECT * FROM cones ORDER BY Price;
```
<hr>
```
sqlite> select * from cones order by Price;
1|strawberry|pink|3.55
2|chocolate|light brown|4.75
5|bubblegum|pink|4.75
3|chocolate|dark brown|5.25
4|strawberry|pink|5.25
6|chocolate|dark brown|5.25
sqlite> 6|chocolate|dark brown|5.25
```

# Selecting columns - Projection

By default, operations on tables refer to *all* the columns.
One can *select* certain columns however, by naming out the specific column in the SELECT statement (instead of using a `*`).

```
SELECT <columns expr> FROM <table>;
```

In [6]:
cones.select(['Flavor', 'Price'])

## SELECT from table

The `SELECT` statement specifies the table explicitly.  (It's not a method on an object.  The database keeps track of all the tables.)

Specify the columns to select

```
SELECT <column names or expr> FROM <table name>;
```
<hr>
```
sqlite> .headers on
sqlite> select Flavor, Price from cones;
Flavor|Price
strawberry|3.55
chocolate|4.75
chocolate|5.25
strawberry|5.25
bubblegum|4.75
chocolate|5.25

```

We can also do expression selection, essentially doing math in our SELECT statement. For example, let's say we want the prices to double in our table:

```
sqlite> .headers on
sqlite> select Flavor, 2 * Price from cones;
Flavor|Price
strawberry|7.10
chocolate|9.50
chocolate|10.50
strawberry|10.50
bubblegum|9.50
chocolate|10.50

```


## Permanent data

Quit sqlite3 with the `.quit` pseudo command. Then start it up again on the same database.  Notice how everything is still there.

## Filtering rows - where

In [7]:
# Here's the simple equality filter in Tables

cones.where('Flavor', 'chocolate')

### SQL where clause

By default, `SELECT` operations on tables refer to *all* the rows.
One can operate on a subset of the rows by specifying a predicate expression in the `WHERE` clause.

```
SELECT <column names or expr> 
FROM <table name>
WHERE <condition>;
```
<hr>

```
sqlite> select * from cones where Flavor = "chocolate";
ID|Flavor|Color|Price
2|chocolate|light brown|4.75
3|chocolate|dark brown|5.25
6|chocolate|dark brown|5.25
```

In [8]:
# Or an expression that produces a boolean array
cones.where(cones["Price"] > 5)

### SQL:

```
sqlite> select * from cones where Price > 5;
ID|Flavor|Color|Price
3|chocolate|dark brown|5.25
4|strawberry|pink|5.25
6|chocolate|dark brown|5.25
```

### Approximate match

Regular expression matches are so common that they are built in in SQL.

```
sqlite> select * from cones where Flavor like "%berry%";
Flavor|Color|Price
strawberry|pink|3.55
strawberry|pink|5.25
sqlite> 
```

## SQL Boolean operators

There are many operators that you can use in SQL.
An operator is a word or character that is used in an SQL statement to perform operation(s), like comparisons and arithmetic operations.

SQLite understands the following binary operators, in order from highest to lowest precedence:
```
||
*    /    %
+    -
<<   >>   &    |
<    <=   >    >=
=    ==   !=   <>   IS   IS NOT   IN   LIKE   GLOB   MATCH   REGEXP
AND   
OR
```
Supported unary prefix operators are these:
```
-    +    ~    NOT
```

# Grouping and Aggregation

The  GROUP BY clause is used to group rows of SELECT statement into a set of summary rows or groups based on values of columns or expressions. 

Apply an aggregate function, such as SUM, AVG, MIN, MAX or COUNT, to each group to output the summary information.

```
SELECT <expr> 
FROM <table name> 
WHERE <cond> 
GROUP BY <grouping>;
```
<hr>

You have been applying aggregation functions to "groups" of rows.  

In [9]:
# By default Tables aggregate with count
cones.group('Flavor')

The analogous SQL has a `group by` clause in the select statement.  The value expression specifies what aggregation operation to perform on the groups.

```
sqlite> select count(Price), Flavor from cones group by Flavor;
count(Price)|Flavor
1|bubblegum
2|chocolate
2|strawberry
```

In [10]:
# You can specify how to aggregate
cones.group('Flavor', min)

In [11]:
# And select the meaningful columns
cones.select(['Flavor', 'Price']).group('Flavor', np.mean)

### SQL GROUP BY built in operators in the column expression

```
sqlite> select min(Price), Flavor from cones group by Flavor;
min(Price)|Flavor
4.75|bubblegum
4.75|chocolate
3.55|strawberry

sqlite> select avg(Price), Flavor from cones group by Flavor;
avg(Price)|Flavor
4.75|bubblegum
5.0|chocolate
4.4|strawberry
sqlite> 
```

### GROUP operators

The following are the most commonly used SQL aggregate functions:

* AVG – calculates the average of a set of values.
* COUNT – counts rows in a specified table or view.
* MIN – gets the minimum value in a set of values.
* MAX – gets the maximum value in a set of values.
* SUM – calculates the sum of values.

Notice that all aggregate functions above ignore NULL values except for the COUNT function.


```
sqlite> .header on
sqlite> select Color, max(Price) - min(Price) from cones group by Color;
Color|max(Price) - min(Price)
dark brown|0.0
light brown|0.0
pink|1.7
```

## Unique values

Being able to determine the unique values in a column or collection of columns in common enough working with data that SQL builds it in.  This is a different sort of aggregation that does not involve groups.

```
select distinct Flavor from cones;
select distinct Flavor, Color from cones;
```

In [12]:
np.unique(cones['Flavor'])

In [13]:
cones.groups(['Flavor', 'Color']).drop('count')

## Joining multiple tables

Information in different tables can be brought together using *join* operations.  You've had a lot of experience with this already.

Let's imagine that there was another table associated with the cashier's transations.

In [14]:
sales = Table(['Cashier', 'Tid']).with_rows([
    ('Baskin', 1),
    ('Robin', 2),
    ('Baskin', 3),
    ('Baskin', 4),
    ('Robin', 5),
    ('Robin', 6)
])
sales

### SQL - create a similar table

```
create table sales as
    select "Baskin" as Cashier, 1 as Tid union
    select "Baskin", 3 union
    select "Baskin", 4 union
    select "Robin", 2 union
    select "Robin", 5 union
    select "Robin", 6;
```
Paste this into sqlite3
```
sqlite> .tables
cones  sales
sqlite> select * from sales;
Cashier|Tid
Baskin|1
Baskin|3
Baskin|4
Robin|2
Robin|5
Robin|6
sqlite> 
```

In [15]:
# The JOIN of these two tables around their related columns
sales.join('Tid', cones, 'Id')

## SQL join 

In SQL the join operation is easily expressed by multiple `FROM` tables, but it is not the same as the "inner join" you are familiar with.

```
SELECT * FROM sales, cones;
```
produces the cross-product (outer join) of all the rows of `sales` and the rows of `cones`.  We want the ones with matching `Id`'s.  This is expressed through the `WHERE` clause.

```
SELECT * FROM sales, cones WHERE Tid=Id;
```

Try it out.

```
sqlite> .tables
cones  sales
sqlite> SELECT * FROM sales, cones WHERE Tid=Id;
Cashier|Tid|Id|Flavor|Color|Price
Baskin|1|1|strawberry|pink|3.55
Baskin|3|3|chocolate|dark brown|5.25
Baskin|4|4|strawberry|pink|5.25
Robin|2|2|chocolate|light brown|4.75
Robin|5|5|bubblegum|pink|4.75
Robin|6|6|chocolate|dark brown|5.25
sqlite> 
```

#### Refining the join description

The names of tables can be used to disambiguate, say if the columns have the same names in different tables.

```
select * from sales, cones where sales.Tid=cones.Id;
```

And, we can tailor the result with the expression.

```
select Flavor, Color, Price, Cashier, Id from sales, cones where sales.Tid=cones.Id;
```

And we could create this as a new, permanent table.

```
create table full as 
    select Flavor, Color, Price, Cashier, ID from sales, cones where sales.Tid=cones.Id;
```

## Creating and dropping tables

```
DROP TABLE <table name>
```
For example, create a new table of the join and drop it.

```
sqlite> create table full as 
   ...>     select Flavor, Color, Price, Cashier, ID from sales, cones where sales.TID=cones.ID;
sqlite> .tables
cones  full   sales
sqlite> select * from full;
Flavor|Color|Price|Cashier|ID
strawberry|pink|3.55|Baskin|1
chocolate|dark brown|5.25|Baskin|3
strawberry|pink|5.25|Baskin|4
chocolate|light brown|4.75|Robin|2
bubblegum|pink|4.75|Robin|5
chocolate|dark brown|5.25|Robin|6
```

```
sqlite> drop table full;
sqlite> .tables
cones  sales
```

## Naming result columns

```
SELECT <expr> as <name> FROM <tables> WHERE <c-expr> GROUP BY <g-expr>

```
Create a table with a `Taste` column with two values: "delicious" and "other".

```
select "delicious" as Taste, Flavor, Color from cones where Flavor is "chocolate" union
select "other", Flavor, Color from cones where Flavor is not "chocolate";
```

# Programming language APIs

Database tables are seldom accessed from a command line as we've done in our interactive session.  They are used by passing queries to the database interpreter - typically underneath some application.  So, they are accessed from within the programming language via an *Application Programming Interface* or *API*.

To learn more about an API and what it is used for, check out [this](https://medium.freecodecamp.org/what-is-an-api-in-english-please-b880a3214a82) link.

Here's an example of the python API to sqlite3.

In [16]:
import sqlite3

In [17]:
icecream = sqlite3.connect('icecream.db')

In [18]:
icecream.execute('SELECT * FROM cones;')

In [19]:
icecream.execute('SELECT DISTINCT Flavor FROM cones;').fetchall()

In [20]:
icecream.execute('SELECT * FROM cones WHERE Flavor is "chocolate";').fetchall()

In [21]:
# Be careful about filling in parameters of the query casually
# as it may be open SQL injection attacks

col = "Flavor"

In [22]:
# This looks like it is providing just the list of flavors

icecream.execute('SELECT DISTINCT %s FROM cones;' % col).fetchall()

In [23]:
# What if an attacker managed this?  Go back and evaluate the query above
col = "*"

In [24]:
# Query parameter substitution allows the parameters to be filled in
# at the database query processor.

icecream.execute('SELECT * FROM cones WHERE Flavor is ?;', ("chocolate",)).fetchall()

In [25]:
# Building abstractions that hide the database
def flavor(db, flv):
    return db.execute('SELECT * FROM cones WHERE Flavor is ?;', (flv,)).fetchall()

In [26]:
flavor(icecream, "chocolate")

## SQL Tables - a new class

The following section is optional and for Python experts (this goes beyond the scope of Data8X programming). Let's derive a class from Tables that allows Tables to be read or written to a database.

This illustrates:
* Class inheritance to extend functionality
* Use of helper functions to make coding clear and abstract messy details of a particular implementation.
* Exception handling to deal with behavior of underlying module
* classmethods and instance methods

In [27]:
# sqlite3 Helper functions for the SQL_Table class

import sqlite3
def build_list(s):
    """Return string representing column name tuple."""
    res = "("
    for v in s[:-1]:
        res = res + v + ", "
    res = res + s[-1] + ")"
    return res

def build_q(n):
    """Return string representing parameter substitution tuple."""
    return "(" + "?, "*(n-1) + "?)"

def sqltable_exists(dbconn, table):
    """Return schema as SQL create statement if table exists."""
    create = dbconn.execute('select sql from sqlite_master where name=?;', 
                          (table,)).fetchall()
    return create[0][0] if create else False

def sqlcol_names(dbconn, table):
    """Return list of column names of a SQL table."""
    cursor = dbconn.execute('SELECT * FROM %s LIMIT 1;' % table)
    col_names = [col[0] for col in cursor.description]
    return col_names

def sqlexec(dbconn, query, verbose=False):
    """Wrap verbose processing around SQL query."""
    if verbose:
        print(query)
    return dbconn.execute(query)

### SQL_Table Class

In [28]:
class SQL_Table(Table):
    """ Extend Table class with methods to read/write a Table
    from/to a table in a SQLite3 database.
    """
    @classmethod
    def read(cls, filepath, table, verbose=False):
        """Create a SQL_Table by reading a table from a SQL database."""
        
        dbconn = sqlite3.connect(filepath, 
                                 detect_types=sqlite3.PARSE_COLNAMES)
        
        col_names = sqlcol_names(dbconn, table)
        rows = sqlexec(dbconn,'SELECT * from %s;' % table, verbose).fetchall()
        dbconn.close()
        return cls(col_names).with_rows(rows)
    
    def write(self, filepath, table, verbose=False, overwrite=True):
        """Write a Table into a SQL database as a SQL table."""
        
        dbconn = sqlite3.connect(filepath)
        # If a table of this name exists, set it aside
        old_table = sqltable_exists(dbconn, table)
        if overwrite and old_table:
            backup = table+"_bkp"
            if sqltable_exists(dbconn, backup):
                sqlexec(dbconn, 'DROP TABLE %s;' % backup, verbose)
            sqlexec(dbconn, 'ALTER TABLE %s RENAME TO %s;' % (table, backup), verbose)

        # Create table and insert each row
        cols = build_list(self.labels)
        sqlexec(dbconn, "CREATE TABLE %s %s;" % (table, cols), verbose)
        for row in self.rows:  
            sqlexec(dbconn, 'INSERT INTO %s VALUES %s;' % (table, tuple(row)), verbose)
        dbconn.commit()
        
        # clean up table, if was set aside
        if overwrite and old_table:
            try:
                sqlexec(dbconn, 'DROP TABLE %s;' % backup, verbose)
            except:
                print('Unable to drop %s', backup)
        dbconn.close()
        
    @classmethod
    def cast(cls, table):
        """Return a SQL_Table version of a Table."""
        return cls().with_columns(zip(table.labels, table.columns))

In [29]:
mycones = SQL_Table.read("icecream.db", "cones", verbose=True)
mycones

In [30]:
type(mycones)

In [31]:
db = mycones.write("test.db", "cones", verbose=True)

In [32]:
SQL_Table.read("test.db", "cones")

In [33]:
scones = SQL_Table.cast(cones)
scones.write("cones.db", "cones")

In [34]:
SQL_Table.read("cones.db", "cones")

## A larger example - NBA Players
We are now going to use a different dataset for more examples. This data set is related to NBA Players and their salary and has information such as the player name, the position they play, what team they are on, and their salary. 

In [35]:
# Table constructor method

# NBA player data from 15-16 season
nba = SQL_Table.read_table('nba_salaries.csv')
nba

SQL equivalent

sqlite3 non-standard way to create a table from a csv file

```
.mode csv
.import nba_salaries.csv nba_raw
```

Now we have a real table to work with
```
sqlite>.tables
nba
sqlite> .headers on
sqlite> select * from nba_raw limit 10;
```

Caution: Here we have specified `limit` to reduce the size of the output.  Unlike the `__repr__` of `Table`, this is actually limiting the size of the returned table value. In non-interactive use, you would seldom want to do this.

In [36]:
nba.write("nba_salaries.db", "nba")

In [37]:
nba_db = sqlite3.connect("nba_salaries.db")

In [38]:
nba.select(['PLAYER', 'POSITION', 'TEAM'])

SQL equivalent

Specify columns by sequence of names separated by ",", instead of "*".

```
select PLAYER, POSITION, TEAM from nba_raw limit 10;

sqlite> select PLAYER, POSITION, TEAM from nba limit 10;
PLAYER,POSITION,TEAM
"Paul Millsap",PF,"Atlanta Hawks"
"Al Horford",C,"Atlanta Hawks"
"Tiago Splitter",C,"Atlanta Hawks"
"Jeff Teague",PG,"Atlanta Hawks"
"Kyle Korver",SG,"Atlanta Hawks"
"Thabo Sefolosha",SF,"Atlanta Hawks"
"Mike Scott",PF,"Atlanta Hawks"
"Kent Bazemore",SF,"Atlanta Hawks"
"Dennis Schroder",PG,"Atlanta Hawks"
"Tim Hardaway Jr.",SG,"Atlanta Hawks"
sqlite> 
```

In [39]:
nba_db.execute("select PLAYER, POSITION, TEAM from nba;").fetchall()

In [40]:
nba.where('TEAM', 'Atlanta Hawks')

SQL equivalent

```
select * from nba_raw where TEAM = "Atlanta Hawks";
```


In [41]:
nba_db.execute('select * from nba where TEAM = "Atlanta Hawks";').fetchall()

In [42]:
nba.where(nba['SALARY'] > 15)

In [43]:
nba_db.execute('select PLAYER, SALARY from nba where SALARY > 15').fetchall()

## Exploring this in the sqlite3 session

Try this using the nba_raw table in the nba_salaries database imported as above.  You will see some of the common subtleties of data types.

```
select PLAYER, SALARY from nba_raw where SALARY > 10 limit 10;

select PLAYER, SALARY from nba_raw where cast(SALARY as int) > 10 limit 10;
```
Why isn't this working?

At the command line we could do
```
.schema raw_nba
```
But within SQL itself we can get to this information because sqlite3
holds its own information in (hidden) tables.

```
select * from sqlite_master;
select sql from sqlite_master where name="nba_raw";
```
We can create a new table from an existing one and get the type specified in the process.

```
create table nba as
    select PLAYER, POSITION, TEAM, cast(SALARY as REAL) as SALARY from nba_raw;
    
select * from nba where SALARY > 15 order by SALARY;
```

### LIKE operator

```
select * from nba where TEAM like "Atlanta%";
```
Try this out
```
sqlite> select * from nba where TEAM like "Atlanta%";
PLAYER,POSITION,TEAM,SALARY
"Paul Millsap",PF,"Atlanta Hawks",18.671659
"Al Horford",C,"Atlanta Hawks",12.0
"Tiago Splitter",C,"Atlanta Hawks",9.75625
"Jeff Teague",PG,"Atlanta Hawks",8.0
"Kyle Korver",SG,"Atlanta Hawks",5.746479
"Thabo Sefolosha",SF,"Atlanta Hawks",4.0
"Mike Scott",PF,"Atlanta Hawks",3.333333
"Kent Bazemore",SF,"Atlanta Hawks",2.0
"Dennis Schroder",PG,"Atlanta Hawks",1.7634
"Tim Hardaway Jr.",SG,"Atlanta Hawks",1.3045200000000001
"Walter Tavares",C,"Atlanta Hawks",1.0
"Jason Richardson",SG,"Atlanta Hawks",0.947276
"Lamar Patterson",SG,"Atlanta Hawks",0.525093
"Terran Petteway",SG,"Atlanta Hawks",0.525093
sqlite> 
```

In [44]:
nba_db.execute('select * from nba where TEAM like "Atlanta%";').fetchall()

### Further Reading
To learn more about SQL and get some more practice with it, feel free to check out the following links as resources:
* [SQL Zoo](https://sqlzoo.net/)
* [Mode Analytics](https://mode.com/)
* [W3Schools](https://www.w3resource.com/sql/tutorials.php)

Stack Overflow and Google are also great ways to learn more from people who are also working with and learning about SQL as well, definitely use them in your path to learning more about data science and SQL!

---

## Finishing up

**Important submission information:** 
- Be sure to run the tests and verify that they all pass by running the `grader.check_all()` cell below,
- Save your progress by choosing the **Save and Checkpoint** item in the **File** menu, 
- Submit your work by clicking the **Submit** button in the toolbar at the top of notebook. 
- Download a zip file of this notebook by running the last cell below. **Note:** Be sure to run all the tests before exporting so that all images/graphs appear in the exported notebook. 

**Please save before submitting!**

In [45]:
# To double-check your work, the cell below will rerun all of the autograder tests.
grader.check_all()

In [46]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)